In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
from shapely.geometry import Point
def intpt_func(row):
    return Point(row['INTPTLON'], row['INTPTLAT'])

In [3]:
#loading LODES data

county_lodes = pd.read_csv('../data/county_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
county_lodes.h_geocode = county_lodes.h_geocode.apply(lambda x: int(x/1000))
county_lodes.w_geocode = county_lodes.w_geocode.apply(lambda x: int(x/1000))
county_lodes.w_geocode = county_lodes.w_geocode.astype(str)
county_lodes.h_geocode = county_lodes.h_geocode.astype(str)

#loading Hamilton county geodata
county_cbg = pd.read_csv('../data/county_cbg.csv')
county_cbg['intpt'] = county_cbg[['INTPTLAT', 'INTPTLON']].apply(lambda p: intpt_func(p), axis=1)
county_cbg = gpd.GeoDataFrame(county_cbg, geometry=gpd.GeoSeries.from_wkt(county_cbg.geometry))
county_cbg.GEOID = county_cbg.GEOID.astype(str)

In [4]:
#loading residential buildings
res_build = pd.read_csv('../data/county_residential_buildings.csv', index_col=0)
res_build = gpd.GeoDataFrame(res_build, geometry=gpd.GeoSeries.from_wkt(res_build.geometry))
res_build['location'] = res_build.geometry.apply(lambda p: [p.y, p.x])

#loading work buildings
com_build = pd.read_csv('../data/county_work_loc_poi_com_civ.csv', index_col=0)
com_build = gpd.GeoDataFrame(com_build, geometry=gpd.GeoSeries.from_wkt(com_build.geometry))
com_build['location'] = com_build.geometry.apply(lambda p: [p.y, p.x])
com_build = com_build.reset_index()
com_build.GEOID = com_build.GEOID.astype(str)

#loading all buildings (MS dataset)
ms_build = pd.read_csv('../data/county_buildings_MS.csv')
ms_build = gpd.GeoDataFrame(ms_build, geometry=gpd.GeoSeries.from_wkt(ms_build.geo_centers))
ms_build.GEOID = ms_build.GEOID.astype(str)
ms_build['location'] = ms_build.geometry.apply(lambda p: [p.y, p.x])

In [5]:
#aggregating total jobs for each combination of home and work cbg 
county_lodes = county_lodes.groupby(['h_geocode', 'w_geocode']).agg(total_jobs=('total_jobs', sum)).reset_index().merge(county_cbg[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').rename({'geometry':'home_geom'}, axis=1).drop('GEOID', axis=1).merge(county_cbg[['GEOID', 'geometry']], left_on='w_geocode', right_on='GEOID').rename({'geometry':'work_geom'}, axis=1).drop('GEOID', axis=1).sort_values('total_jobs', ascending=False).reset_index(drop=True)
county_lodes = gpd.GeoDataFrame(county_lodes)

In [6]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [7]:
#generating array of start and return times (in 15 min intervals)
from datetime  import datetime, timedelta
times_morning = [datetime.strptime(dt.strftime('%H%M%S'), '%H%M%S') for dt in 
       datetime_range(datetime(2021, 9, 1, 7), datetime(2021, 9, 1, 9, 10), 
       timedelta(seconds=15))]
times_evening = [datetime.strptime(dt.strftime('%H%M%S'), '%H%M%S') for dt in 
       datetime_range(datetime(2021, 9, 1, 16), datetime(2021, 9, 1, 18, 10), 
       timedelta(seconds=15))]

In [8]:
(times_morning[5] - datetime(1900, 1, 1)).total_seconds()
# times_morning[1].strftime('%H:%M:%S')

25275.0

In [9]:
res_build.GEOID = res_build.GEOID.astype(str)
com_build.GEOID = com_build.GEOID.astype(str)

In [11]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

#setting the random seed
np.random.seed(42)
random.seed(42)

prob_matrix = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(county_lodes.iterrows(), total=county_lodes.shape[0]):

    res = res_build[res_build.GEOID == movement.h_geocode].reset_index(drop=True)
    if res.empty:
        try:
            res = ms_build[ms_build.GEOID == movement.h_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)
        except:
            res = ms_build[ms_build.GEOID == movement.h_geocode].sample(n=movement.total_jobs, random_state=42, replace=True).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.w_geocode].reset_index(drop=True)
    if com.empty:
        try:
            com = ms_build[ms_build.GEOID == movement.w_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)
        except:
            com = ms_build[ms_build.GEOID == movement.w_geocode].sample(n=movement.total_jobs, random_state=42, replace=True).reset_index(drop=True)
            
    r = res
    c = com
   
    for job in range(movement.total_jobs):
     
        if c.empty:
            c = com
        if r.empty:
            r = res

        rand_r = random.randrange(0, r.shape[0])
        rand_c = random.randrange(0, c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)
        
        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[job, 'h_geocode'] = movement.h_geocode
        temp.loc[job, 'w_geocode'] = movement.w_geocode
        temp.loc[job, 'total_jobs'] = movement.total_jobs
        temp.loc[job, 'home_loc_lat'] = r_df.location[0]
        temp.loc[job, 'home_loc_lon'] = r_df.location[1]
        temp.loc[job, 'work_loc_lat'] = c_df.location[0]
        temp.loc[job, 'work_loc_lon'] = c_df.location[1]
        temp.loc[job, 'go_time'] = time_slot1[0].time()
        temp.loc[job, 'go_time_secs'] = (time_slot1[0] - datetime(1900, 1, 1)).total_seconds()
        temp.loc[job, 'go_time_str'] = time_slot1[0].strftime('%H:%M:%S')
        temp.loc[job, 'return_time'] = time_slot2[0].time()
        temp.loc[job, 'return_time_secs'] = (time_slot2[0] - datetime(1900, 1, 1)).total_seconds()
        temp.loc[job, 'return_time_str'] = time_slot2[0].strftime('%H:%M:%S')

        prob_matrix = prob_matrix.append(temp, ignore_index=True)

  0%|          | 0/29439 [00:00<?, ?it/s]

In [12]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [13]:
# convert the lat and lon points to shapely Points
prob_matrix['home_geom'] = prob_matrix[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix['work_geom'] = prob_matrix[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)
prob_matrix.h_geocode = prob_matrix.h_geocode.astype(str)
prob_matrix.w_geocode = prob_matrix.w_geocode.astype(str)

In [14]:
prob_matrix.head()

,h_geocode,w_geocode,total_jobs,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,go_time_secs,go_time_str,return_time,return_time_secs,return_time_str,home_geom,work_geom
0,470370165002,470370165002,393.0,36.144515,-86.804675,36.139164,-86.801478,07:25:30,26730.0,07:25:30,17:48:45,64125.0,17:48:45,POINT (-86.80467509713218 36.14451501842736),POINT (-86.80147778059765 36.13916402739309)
1,470370165002,470370165002,393.0,36.147068,-86.804350,36.145383,-86.803053,08:07:30,29250.0,08:07:30,16:26:30,59190.0,16:26:30,POINT (-86.80435042758323 36.14706795997379),POINT (-86.8030527333781 36.14538287515627)
2,470370165002,470370165002,393.0,36.139983,-86.806641,36.144534,-86.800582,07:17:45,26265.0,07:17:45,16:05:00,57900.0,16:05:00,POINT (-86.80664140025164 36.13998250452984),POINT (-86.80058155485116 36.14453437806355)
3,470370165002,470370165002,393.0,36.141190,-86.805937,36.141609,-86.800945,07:30:15,27015.0,07:30:15,17:56:30,64590.0,17:56:30,POINT (-86.80593675735163 36.14119007450109),POINT (-86.80094516529478 36.14160931321479)
4,470370165002,470370165002,393.0,36.145246,-86.805230,36.141086,-86.800813,07:53:30,28410.0,07:53:30,17:22:30,62550.0,17:22:30,POINT (-86.80522989365294 36.14524616558947),POINT (-86.80081320000002 36.14108625)


In [15]:
prob_matrix.to_csv('../data/county_lodes_combinations.csv', index=False)
# prob_matrix.to_parquet('../data/county_lodes_combinations.parquet', engine='pyarrow', index=False)